In [2]:
from atmospheric_explorer.cams_interfaces import InversionOptimisedGreenhouseGas, EAC4Instance
from atmospheric_explorer.shapefile import ShapefilesDownloader
from atmospheric_explorer.utils import get_local_folder
from atmospheric_explorer.units_conversion import convert_units_array
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import os
from glob import glob
import xarray as xr
from datetime import datetime
import geopandas as gpd
from shapely.geometry import mapping
import plotly.graph_objects as go
import shutil
import numpy as np
import statsmodels.stats.api as sms
import pandas as pd
from plotly.subplots import make_subplots
from math import ceil
from atmospheric_explorer.plotting_apis import surface_satellite_yearly_plot, add_ci, clip_and_concat_countries, line_with_ci_subplots
from atmospheric_explorer.utils import hex_to_rgb

INFO|units_conversion: Loaded constants from file constants.cfg
INFO|units_conversion: Evaluating arithmetic formulas in config
DEBUG|units_conversion: Parsing 1
DEBUG|units_conversion: Parsing 1
DEBUG|units_conversion: Parsing 1
DEBUG|units_conversion: Parsing 1
DEBUG|units_conversion: Parsing 1
DEBUG|units_conversion: Parsing 1
DEBUG|units_conversion: Parsing 1
DEBUG|units_conversion: Parsing 1
DEBUG|units_conversion: Parsing 1
DEBUG|units_conversion: Parsing 1
DEBUG|units_conversion: Parsing 1
DEBUG|units_conversion: Parsing 1
DEBUG|units_conversion: Parsing 1
DEBUG|units_conversion: Parsing 1
DEBUG|units_conversion: Parsing 1
DEBUG|units_conversion: Parsing 1
DEBUG|units_conversion: Parsing 1
DEBUG|units_conversion: Parsing 1
DEBUG|units_conversion: Parsing 1
DEBUG|units_conversion: Parsing 1
DEBUG|units_conversion: Parsing 1
DEBUG|units_conversion: Parsing 6.0221415e23*1e-18/(1e4*28.01e-3)
DEBUG|units_conversion: Parsing 1
DEBUG|units_conversion: Parsing 2004008
DEBUG|units_conver

In [3]:
# Remove previous data
shutil.rmtree(os.path.join(get_local_folder(), 'data'))

In [4]:
# Remove previous data
shutil.rmtree(os.path.join(get_local_folder(), 'shapefiles'))

In [7]:
years=[str(y) for y in range(2010, 2021)]
months=[
    '01', '02', '03',
    '04', '05', '06',
    '07', '08', '09',
    '10', '11', '12'
]
countries = ['Italy', 'Russia', 'France', 'Spain', 'Germany', 'Finland']

In [8]:
surface_satellite_yearly_plot("carbon_dioxide", countries, years, months, "test")

INFO|cams_interfaces: Created folder /home/luigibrancati/.atmospheric_explorer/data/global_greenhouse_gas_inversion
INFO|cams_interfaces: Created folder /home/luigibrancati/.atmospheric_explorer/data/global_greenhouse_gas_inversion/data_3
INFO|cams_interfaces: Finished downloading file /home/luigibrancati/.atmospheric_explorer/data/global_greenhouse_gas_inversion/data_3/data_3.zip
INFO|cams_interfaces: Extracted file /home/luigibrancati/.atmospheric_explorer/data/global_greenhouse_gas_inversion/data_3/data_3.zip to folder /home/luigibrancati/.atmospheric_explorer/data/global_greenhouse_gas_inversion/data_3
INFO|cams_interfaces: Updated file_full_path to wildcard path /home/luigibrancati/.atmospheric_explorer/data/global_greenhouse_gas_inversion/data_3/*.nc
INFO|cams_interfaces: Removed /home/luigibrancati/.atmospheric_explorer/data/global_greenhouse_gas_inversion/data_3/data_3.zip
INFO|cams_interfaces: Created folder /home/luigibrancati/.atmospheric_explorer/data/global_greenhouse_gas_

In [ ]:
surface_data = InversionOptimisedGreenhouseGas(
    data_variables="carbon_dioxide",
    file_format="zip",
    quantity="surface_flux",
    input_observations="surface",
    time_aggregation="monthly_mean",
    year=years,
    month=months,
)
surface_data.download()
satellite_data = InversionOptimisedGreenhouseGas(
    data_variables="carbon_dioxide",
    file_format="zip",
    quantity="surface_flux",
    input_observations="satellite",
    time_aggregation="monthly_mean",
    year=years,
    month=months,
)
satellite_data.download()

In [ ]:
# Read data as dataset
df_surface = surface_data.read_dataset(var_name="flux_foss")
df_satellite = satellite_data.read_dataset(var_name="flux_foss")

In [ ]:
df_total = xr.concat([df_surface, df_satellite], dim="input_observations").squeeze()
df_total = df_total.rio.write_crs("EPSG:4326")
# Clip countries
df_total = clip_and_concat_countries(df_total, countries)
# Drop all values that are null over all coords, compute the mean of the remaining values over long and lat
df_total = df_total.sortby("time").mean(dim=["longitude", "latitude"])

In [ ]:
# Convert units
da_converted = convert_units_array(df_total["flux_foss"], "carbon_dioxide")
da_converted.name = df_total["flux_foss"].name
unit = da_converted.attrs["units"]

In [ ]:
import xarray as xr, pandas as pd, numpy as np

x = np.linspace(-110, -90, 5)
y = np.linspace(23, 30, 5)
time = pd.date_range('1990-01-01', '2100-12-01', freq='MS')

da = xr.DataArray(
    np.random.random(size=(5, 5, len(time))),
    dims=['x', 'y', 'time'],
    coords=[x, y, time],
    name='temperature',
)

In [ ]:
from functools import singledispatch

In [ ]:
@singledispatch
def confidence_interval(array: list | np.ndarray) -> tuple:
    lower, upper = sms.DescrStatsW(array).tconfint_mean()
    return lower, np.mean(array), upper

@confidence_interval.register
def _(array: xr.DataArray, dim: str) -> xr.DataArray:
    all_dims = list(array.dims)
    index = all_dims.index(dim)
    keep_dims = all_dims
    keep_dims.remove(dim)
    return xr.DataArray(
        np.apply_along_axis(confidence_interval, index, array.values),
        dims=[*keep_dims, 'ci'],
        coords=[*[array.coords[d] for d in keep_dims], ['lower', 'mean', 'upper']]
    )

In [ ]:
confidence_interval(da, 'time')

In [ ]:
da_converted_agg = da_converted.resample(time='YS', keep_attrs=True).map(confidence_interval, dim='time')
da_converted_agg.name = da_converted.name
da_converted_agg.attrs = da_converted.attrs

In [ ]:
da_converted

In [ ]:
da_converted_agg

In [ ]:
fig = line_with_ci_subplots(da_converted_agg.to_dataframe().unstack('ci').droplevel(axis=1, level=0).reset_index(['admin', 'input_observations']), countries, 2, unit, "test")
fig.show()